**Zadanie 1**

Uruchom przykład Incremental learning z punktu 2.1 dobierając parametry tak, aby ilość danych do przeliczenia była większa niż sumaryczna ilość pamięci RAM workerów. Obserwuj daszboard i w razie niepowodzenia dostosuj wielkość i ilość chunków tak, aby obliczenia się wykonały na tych samych parametrach workerów. Zobacz jak wygląda struktura pamięci na workerach, czy nie dochodzi do zrzucania pamięci na dysk (zapewne będzie on wąskim gardłem, więc w menedżerze będzie widać jego mocne obciążenie). Zastanów się czy można to jakoś zoptymalizować przy dostępnych workerach i wykonaj kilka eksperymentów szukając większej wydajności i krótszego czasu wykonania całego zadania.

In [3]:
import numpy as np
import dask.array as da
from dask_ml.model_selection import KFold
import xgboost as xgb
from dask.distributed import LocalCluster

X = np.random.rand(1000000, 100)
y = np.random.randint(0, 7, 1000000)

X = da.from_array(X, chunks=(10000, 100))
y = da.from_array(y, chunks=(10000,))

In [4]:
params = {
    'objective': 'multi:softmax',
    'max_depth': 4,
    'eta': 0.01,
    'subsample': 0.5,
    'min_child_weight': 0.5,
    'num_class': 7
}

n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True, random_state=94)

predictions = {}

with LocalCluster(n_workers=4, threads_per_worker=2, memory_limit="4GB") as cluster:
    display(cluster)
    with cluster.get_client() as client:

        for i, (train, test) in enumerate(cv.split(X, y)):

            X_train = X[train, :-1]
            X_test = X[test, :-1]
            y_train = y[train]
            y_test = y[test]

            d_train = xgb.dask.DaskDMatrix(client, X_train, y_train, enable_categorical=True)
            model = xgb.dask.train(client, params=params, dtrain=d_train)
            predictions[f'fold_{i}'] = xgb.dask.predict(client, model, X_test)

LocalCluster(976fb454, 'tcp://127.0.0.1:55232', workers=4, threads=8, memory=14.90 GiB)

c:\Users\karol\GitHub\analiza_duzych_zbiorow_danych\.venv\Lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| numpy   | 2.1.2  | 2.1.2     | 2.0.2   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
c:\Users\karol\GitHub\analiza_duzych_zbiorow_danych\.venv\Lib\site-packages\dask\array\slicing.py:987: PerformanceWarning: Increasing number of chunks by factor of 80
  p = blockwise(
c:\Users\karol\GitHub\analiza_duzych_zbiorow_danych\.venv\Lib\site-packages\dask\array\slicing.py:987: PerformanceWarning: Increasing number of chunks by factor of 20
  p = blockwise(
Windows is not officially supported for dask/xgboost, contribution are welcomed.
c:\Users\karol\GitHub\analiza_duzych_zbiorow_danych\.venv\Lib\site-packages\distributed\client.py:3361: Use

In [5]:
predictions

{'fold_0': dask.array<mapped_predict, shape=(200000,), dtype=float32, chunksize=(10000,), chunktype=numpy.ndarray>,
 'fold_1': dask.array<mapped_predict, shape=(200000,), dtype=float32, chunksize=(10000,), chunktype=numpy.ndarray>,
 'fold_2': dask.array<mapped_predict, shape=(200000,), dtype=float32, chunksize=(10000,), chunktype=numpy.ndarray>,
 'fold_3': dask.array<mapped_predict, shape=(200000,), dtype=float32, chunksize=(10000,), chunktype=numpy.ndarray>,
 'fold_4': dask.array<mapped_predict, shape=(200000,), dtype=float32, chunksize=(10000,), chunktype=numpy.ndarray>}

**Zadanie 2**

Dokonaj serializacji modelu z zadania 1 na dysk i następnie go wczytaj ponownie tak, aby można było uruchomić na nim predykcję dla tablic X_test oraz y_test (dla użycia miar klasyfikacji) i wyświetl macierz klasyfikacji (confusion matrix).

In [11]:
from sklearn.datasets import fetch_rcv1, fetch_covtype
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, cross_validate
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

import numpy as np
import dask.array as da
import os
import xgboost as xgb
import fastparquet
import pickle

cov = fetch_covtype()
X, y = cov.data, cov.target
data = [X, da.atleast_2d(y).T]
ddf = da.concatenate(data, axis=1).to_dask_dataframe(columns=cov.feature_names + cov.target_names, index=None).compute()
ddf = ddf.reset_index()
DATADIR = './data/'
ddf.to_parquet(os.path.join(DATADIR, 'data.parquet'))
le = LabelEncoder()
y = le.fit_transform(y)

n_splits = 5
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score


cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=94)
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=3)

results = {}

# będziemy zapisywać model
model_dir = "./models/"

for train, test in tqdm(cv.split(X, y), desc="Training...", total=n_splits):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    est, train_score, test_score = fit_and_score(
        clone(clf), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)

# for fold_n, fold in enumerate(results.items(), start=1):
#     print(f"Fold {fold_n}: train_score: {fold[1][0]}, test_score: {fold[1][1]}")

for fold_n, (est, scores) in enumerate(results.items(), start=1):
    model_path = os.path.join(model_dir, f"xgboost_model_fold_{fold_n}.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(est, f)
    print(f"Model dla folda {fold_n} zapisany w: {model_path}")

model_path = os.path.join(model_dir, "xgboost_model_fold_1.pkl")
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

train_index, test_index = list(cv.split(X, y))[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
y_pred = loaded_model.predict(X_test)

cm = confusion_matrix(y[y_test], y_pred)
print("Confusion Matrix:")
print(cm)

[0]	validation_0-mlogloss:1.42195
[1]	validation_0-mlogloss:1.17182
[2]	validation_0-mlogloss:1.00835
[3]	validation_0-mlogloss:0.89389
[4]	validation_0-mlogloss:0.81015
[5]	validation_0-mlogloss:0.74716
[6]	validation_0-mlogloss:0.69977
[7]	validation_0-mlogloss:0.66108
[8]	validation_0-mlogloss:0.63163
[9]	validation_0-mlogloss:0.60702
[10]	validation_0-mlogloss:0.58751
[11]	validation_0-mlogloss:0.56978
[12]	validation_0-mlogloss:0.55626
[13]	validation_0-mlogloss:0.54217
[14]	validation_0-mlogloss:0.52944
[15]	validation_0-mlogloss:0.52098
[16]	validation_0-mlogloss:0.51338
[17]	validation_0-mlogloss:0.50204
[18]	validation_0-mlogloss:0.49490
[19]	validation_0-mlogloss:0.48963
[20]	validation_0-mlogloss:0.48454
[21]	validation_0-mlogloss:0.48032
[22]	validation_0-mlogloss:0.47495
[23]	validation_0-mlogloss:0.47029
[24]	validation_0-mlogloss:0.46705
[25]	validation_0-mlogloss:0.46266
[26]	validation_0-mlogloss:0.45989
[27]	validation_0-mlogloss:0.45814
[28]	validation_0-mlogloss:0.4

[0]	validation_0-mlogloss:1.42176
[1]	validation_0-mlogloss:1.17143
[2]	validation_0-mlogloss:1.00867
[3]	validation_0-mlogloss:0.89337
[4]	validation_0-mlogloss:0.81032
[5]	validation_0-mlogloss:0.74760
[6]	validation_0-mlogloss:0.69993
[7]	validation_0-mlogloss:0.66248
[8]	validation_0-mlogloss:0.63293
[9]	validation_0-mlogloss:0.60745
[10]	validation_0-mlogloss:0.58742
[11]	validation_0-mlogloss:0.56994
[12]	validation_0-mlogloss:0.55458
[13]	validation_0-mlogloss:0.54131
[14]	validation_0-mlogloss:0.53070
[15]	validation_0-mlogloss:0.52299
[16]	validation_0-mlogloss:0.51322
[17]	validation_0-mlogloss:0.50385
[18]	validation_0-mlogloss:0.49852
[19]	validation_0-mlogloss:0.49282
[20]	validation_0-mlogloss:0.48832
[21]	validation_0-mlogloss:0.48162
[22]	validation_0-mlogloss:0.47751
[23]	validation_0-mlogloss:0.47346
[24]	validation_0-mlogloss:0.47022
[25]	validation_0-mlogloss:0.46638
[26]	validation_0-mlogloss:0.46355
[27]	validation_0-mlogloss:0.46158
[28]	validation_0-mlogloss:0.4

[0]	validation_0-mlogloss:1.42116
[1]	validation_0-mlogloss:1.17126
[2]	validation_0-mlogloss:1.00631
[3]	validation_0-mlogloss:0.89097
[4]	validation_0-mlogloss:0.80859
[5]	validation_0-mlogloss:0.74493
[6]	validation_0-mlogloss:0.69652
[7]	validation_0-mlogloss:0.65983
[8]	validation_0-mlogloss:0.62964
[9]	validation_0-mlogloss:0.60545
[10]	validation_0-mlogloss:0.58556
[11]	validation_0-mlogloss:0.56730
[12]	validation_0-mlogloss:0.55354
[13]	validation_0-mlogloss:0.54053
[14]	validation_0-mlogloss:0.53027
[15]	validation_0-mlogloss:0.52081
[16]	validation_0-mlogloss:0.51157
[17]	validation_0-mlogloss:0.50438
[18]	validation_0-mlogloss:0.49719
[19]	validation_0-mlogloss:0.49104
[20]	validation_0-mlogloss:0.48445
[21]	validation_0-mlogloss:0.48106
[22]	validation_0-mlogloss:0.47541
[23]	validation_0-mlogloss:0.47202
[24]	validation_0-mlogloss:0.46831
[25]	validation_0-mlogloss:0.46543
[26]	validation_0-mlogloss:0.46306
[27]	validation_0-mlogloss:0.46088
[28]	validation_0-mlogloss:0.4

[0]	validation_0-mlogloss:1.42191
[1]	validation_0-mlogloss:1.17121
[2]	validation_0-mlogloss:1.00664
[3]	validation_0-mlogloss:0.89184
[4]	validation_0-mlogloss:0.80911
[5]	validation_0-mlogloss:0.74673
[6]	validation_0-mlogloss:0.69777
[7]	validation_0-mlogloss:0.65979
[8]	validation_0-mlogloss:0.62943
[9]	validation_0-mlogloss:0.60493
[10]	validation_0-mlogloss:0.58421
[11]	validation_0-mlogloss:0.56701
[12]	validation_0-mlogloss:0.55301
[13]	validation_0-mlogloss:0.53867
[14]	validation_0-mlogloss:0.52816
[15]	validation_0-mlogloss:0.51994
[16]	validation_0-mlogloss:0.51138
[17]	validation_0-mlogloss:0.50445
[18]	validation_0-mlogloss:0.49721
[19]	validation_0-mlogloss:0.49259
[20]	validation_0-mlogloss:0.48386
[21]	validation_0-mlogloss:0.48001
[22]	validation_0-mlogloss:0.47449
[23]	validation_0-mlogloss:0.46976
[24]	validation_0-mlogloss:0.46668
[25]	validation_0-mlogloss:0.46263
[26]	validation_0-mlogloss:0.46049
[27]	validation_0-mlogloss:0.45535
[28]	validation_0-mlogloss:0.4

[0]	validation_0-mlogloss:1.42224
[1]	validation_0-mlogloss:1.17167
[2]	validation_0-mlogloss:1.00765
[3]	validation_0-mlogloss:0.89272
[4]	validation_0-mlogloss:0.81038
[5]	validation_0-mlogloss:0.74764
[6]	validation_0-mlogloss:0.69840
[7]	validation_0-mlogloss:0.66096
[8]	validation_0-mlogloss:0.63088
[9]	validation_0-mlogloss:0.60588
[10]	validation_0-mlogloss:0.58459
[11]	validation_0-mlogloss:0.56682
[12]	validation_0-mlogloss:0.55121
[13]	validation_0-mlogloss:0.53658
[14]	validation_0-mlogloss:0.52553
[15]	validation_0-mlogloss:0.51434
[16]	validation_0-mlogloss:0.50654
[17]	validation_0-mlogloss:0.49857
[18]	validation_0-mlogloss:0.49273
[19]	validation_0-mlogloss:0.48694
[20]	validation_0-mlogloss:0.48286
[21]	validation_0-mlogloss:0.47840
[22]	validation_0-mlogloss:0.47374
[23]	validation_0-mlogloss:0.47045
[24]	validation_0-mlogloss:0.46548
[25]	validation_0-mlogloss:0.46212
[26]	validation_0-mlogloss:0.46021
[27]	validation_0-mlogloss:0.45615
[28]	validation_0-mlogloss:0.4



Training...: 100%|██████████| 5/5 [00:56<00:00, 11.35s/it]


Model dla folda 1 zapisany w: ./models/xgboost_model_fold_1.pkl
Model dla folda 2 zapisany w: ./models/xgboost_model_fold_2.pkl
Model dla folda 3 zapisany w: ./models/xgboost_model_fold_3.pkl
Model dla folda 4 zapisany w: ./models/xgboost_model_fold_4.pkl
Model dla folda 5 zapisany w: ./models/xgboost_model_fold_5.pkl
Confusion Matrix:
[[    0     0     0     0     0     0     0]
 [    7   462  7051   535     6  3113     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [41263 58066   272     0  1275   188  3965]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]]
